In [17]:
import json  
from pprint import pprint

from evm.rlp.headers import (
    BlockHeader,
    CollationHeader,
)

from evm.db.chain import ChainDB 
from evm import db
 
from evm.vm.forks.tangerine_whistle.vm_state import TangerineWhistleVMState  
from evm.vm.forks import TangerineWhistleVM 

from evm.tools import fixture_tests 
from eth_keys import keys
from evm.exceptions import ValidationError


In [18]:
transaction_string = '''
{
    "env" : 
    {
        "currentCoinbase" : "0x2adc25665018aa1fe0e6bc666dac8fc2697ff9ba",
        "currentDifficulty" : "0x20000",
        "currentGasLimit" : "0x174876e800",
        "currentNumber" : "0x01",
        "currentTimestamp" : "0x03e8",
        "previousHash" : "0x5e20a0453cecd065ea59c37ac63e079ee08998b6045136a8ce6635c7912ec0b6"
    },
    "EIP150" : {
        "hash" : "0x7e7ff82e1c33ffbeb3dedb679498c9716f83c697f497d75aca467ae936b8ae45",
        "indexes" : {
            "data" : 0,
            "gas" : 0,
            "value" : 0
        },
        "logs" : "0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347"
    }, 
    "pre" : {
    "0xa94f5374fce5edbc8e2a8697c15331677e6ebf0b" : {
                "balance" : "0xe8d4a51000",
                "code" : "",
                "nonce" : "0x00",
                "storage" : {
                }
            },
            "0x6a0a0fc761c612c340a0e98d33b37a75e5268472" : {
                "balance" : "0x0de0b6b3a7640000",
                "code" : "0x7f6004600c60003960046000f3600035ff00000000000000000000000000000000600052602060006000f0600054805b6001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1506001018060005260008060208180876006f1505a616000106200002f57600055",
                "nonce" : "0x00",
                "storage" : {
                }
            }
    },
    "transaction" : {
        "data" : [
            ""
        ],
        "gasLimit" : [
            "0x0c3500"
        ],
        "gasPrice" : "0x01",
        "nonce" : "0x00",
        "secretKey" : "0x45a915e4d060149eb4365960e6a7a45f334393093061116b197e3240065ff2d8",
        "to" : "0x6a0a0fc761c612C340a0e98d33b37a75e5268472",
        "value" : [
            "0x00"
        ]
    }
} 
'''

transaction_obj = json.loads(transaction_string) 

In [38]:
def normalize_state(obj):  
    normalized = {
        'env': fixture_tests.normalize_environment(obj['env']),
        'pre': fixture_tests.normalize_account_state(obj['pre']), 
        'post': fixture_tests.normalize_post_state(obj['EIP150']),
        'transaction': fixture_tests.normalize_unsigned_transaction(obj['transaction'],obj["EIP150"]['indexes'])
    } 
    return normalized
normalize_obj  = normalize_state(transaction_obj)
normalize_obj

{'env': {'currentCoinbase': b'*\xdc%fP\x18\xaa\x1f\xe0\xe6\xbcfm\xac\x8f\xc2i\x7f\xf9\xba',
  'currentDifficulty': 131072,
  'currentGasLimit': 100000000000,
  'currentNumber': 1,
  'currentTimestamp': 1000,
  'previousHash': b'^ \xa0E<\xec\xd0e\xeaY\xc3z\xc6>\x07\x9e\xe0\x89\x98\xb6\x04Q6\xa8\xcef5\xc7\x91.\xc0\xb6'},
 'post': {'hash': b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE',
  'logs': b'\x1d\xccM\xe8\xde\xc7]z\xab\x85\xb5g\xb6\xcc\xd4\x1a\xd3\x12E\x1b\x94\x8at\x13\xf0\xa1B\xfd@\xd4\x93G'},
 'pre': {b'j\n\x0f\xc7a\xc6\x12\xc3@\xa0\xe9\x8d3\xb3zu\xe5&\x84r': {'balance': 1000000000000000000,
   'code': b'\x7f`\x04`\x0c`\x009`\x04`\x00\xf3`\x005\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00R` `\x00`\x00\xf0`\x00T\x80[`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x81\x80\x87`\x06\xf1P`\x01\x01\x80`\x00R`\x00\x80` \x8

In [20]:
def get_ancestor_block_hash(self, block_number):
    if block_number >= self.block_number:
        return b''
    elif block_number < 0:
        return b''
    elif block_number < self.block_number - 256:
        return b''
    else:
        return keccak(text="{0}".format(block_number))

def get_prev_hashes(self, last_block_hash, db):
    return [] 

In [21]:
_tangerineWhistleVMState = TangerineWhistleVMState.configure(
    get_ancestor_hash=get_ancestor_block_hash
)

In [22]:
_tangerineWhistleVM = TangerineWhistleVM.configure( 
    _state_class=_tangerineWhistleVMState,
    get_prev_hashes=get_prev_hashes,
)

In [23]:
header = BlockHeader(
    coinbase=normalize_obj['env']['currentCoinbase'],
    difficulty=normalize_obj['env']['currentDifficulty'],
    block_number=normalize_obj['env']['currentNumber'],
    gas_limit=normalize_obj['env']['currentGasLimit'],
    timestamp=normalize_obj['env']['currentTimestamp'],
    parent_hash=normalize_obj['env']['previousHash']
)
header

<BlockHeader #1 e8682bfd>

In [24]:
chaindb = ChainDB(db.get_db_backend()) #defult memeory db

In [25]:
vm = _tangerineWhistleVM(header=header, chaindb=chaindb)

In [26]:
#fail
# with vm.state.mutable_state_db() as state_db: 
#     state_db.apply_state_dict(normalize_obj['pre']) 
# vm.block.header.state_root = vm.state.state_root 

In [27]:
#work
vm_state = vm.state
with vm_state.mutable_state_db() as state_db: 
    state_db.apply_state_dict(normalize_obj['pre']) 
vm.block.header.state_root = vm_state.state_root

In [28]:
unsigned_transaction = vm.create_unsigned_transaction(
        nonce=normalize_obj['transaction']['nonce'],
        gas_price=normalize_obj['transaction']['gasPrice'],
        gas=normalize_obj['transaction']['gasLimit'],
        to=normalize_obj['transaction']['to'],
        value=normalize_obj['transaction']['value'],
        data=normalize_obj['transaction']['data'],
    ) 

In [29]:
private_key = keys.PrivateKey(normalize_obj['transaction']['secretKey']) 
transaction = unsigned_transaction.as_signed_transaction(private_key=private_key) 

In [30]:
try:
    computation, _ = vm.apply_transaction(transaction) 
except ValidationError as transaction_error :
    print(transaction_error)
else:
    transaction_error = False 

In [40]:
print(vm.block.header.state_root)
print(normalize_obj['post']['hash'])
print(vm.block.header.state_root == normalize_obj['post']['hash']) 

b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE'
b'~\x7f\xf8.\x1c3\xff\xbe\xb3\xde\xdbg\x94\x98\xc9qo\x83\xc6\x97\xf4\x97\xd7Z\xcaFz\xe96\xb8\xaeE'
True
